In [7]:
# Importar las bibliotecas necesarias
import pandas as pd
import sqlite3

# Conectar a la base de datos SQLite
conn = sqlite3.connect('ventas.db')
cursor = conn.cursor()

# Crear la tabla
cursor.execute('''
    CREATE TABLE IF NOT EXISTS ventas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        "Categoría" TEXT,
        "Sucursal" TEXT,
        "Total Vendido" REAL,
        "Ingreso Total" REAL
    )
''')



# Cargar los datos del CSV en la tabla
df = pd.read_csv('resultados_ventas.csv')
df.to_sql('ventas', conn, if_exists='replace', index=False)

# Realizar consultas SQL
print("**Consulta 1: Ventas totales por categoría**")
cursor.execute('''
    SELECT "Categoría", SUM("Total Vendido") AS total_vendido
    FROM ventas
    GROUP BY "Categoría"
''')
resultados = cursor.fetchall()
for resultado in resultados:
    print(resultado)


print("\n**Consulta 2: Sucursal con mayor ingreso total**")
cursor.execute('''
    SELECT "Sucursal", SUM("Ingreso Total") AS ingreso_total
    FROM ventas
    GROUP BY "Sucursal"
    ORDER BY ingreso_total DESC
    LIMIT 1
''')
resultado = cursor.fetchone()
print(resultado)

print("\n**Consulta 3: Categorías con ventas superiores a 1000**")
cursor.execute('''
    SELECT "Categoría", SUM("Total Vendido") AS "total vendido"
    FROM ventas
    GROUP BY "Categoría"
    HAVING SUM("Total Vendido") > 1000
''')
resultados = cursor.fetchall()
for resultado in resultados:
    print(resultado)

# Crear una función para buscar datos
def buscar_datos(categoria=None, sucursal=None):
    query = '''
        SELECT *
        FROM ventas
    '''
    condiciones = []
    parametros = []

    if categoria:
        condiciones.append('"Categoría" = ?')
        parametros.append(categoria)
    if sucursal:
        condiciones.append('"Sucursal" = ?')
        parametros.append(sucursal)

    if condiciones:
        query += ' WHERE ' + ' AND '.join(condiciones)

    cursor.execute(query, parametros)
    resultados = cursor.fetchall()
    return resultados

# Llamar a la función buscar_datos
resultados = buscar_datos(categoria='Electrónicos', sucursal='Sucursal 1')
for resultado in resultados:
    print(resultado)

# Cerrar la conexión
conn.close()

**Consulta 1: Ventas totales por categoría**
('Accesorios', 125)
('Audio', 67)
('Laptop', 29)
('Smartphone', 110)
('Tablet', 41)

**Consulta 2: Sucursal con mayor ingreso total**
('Centro', 37249.22)

**Consulta 3: Categorías con ventas superiores a 1000**
